In [3]:
import pandas as pd
import numpy as np

from datetime import datetime

import sys
sys.path.append("..") 

from tp_config import *
from data_provider import read_prices, read_data,  market_to_symbol
import json

import pandas_ta as ta

In [19]:

def ts_to_datetime(ts):
#datetime.utcfromtimestamp(ts)
    return datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

#Торговля одним контрактом
def simulation_01(prices, df_signals, init_data):
    market          = init_data['market']
    commission      = init_data['exchange_commission'] / 100
    start_capital   = init_data['start_capital']
    buy_correction  = init_data['buy_correction']
    sell_correction = init_data['sell_correction']
    contract        = init_data['contract']
     
    num_shares   = 0
    trade_profit = 0
    cum_profit   = 0    
    market_position = 0
    
    signals = np.zeros(df_signals.shape[0])
    ticks = prices['close'].values
    times = prices['T'].values
    start = len(ticks) - len(signals) 
    ticks = ticks[start:]
    times = times[start:]
    trade_statistics = []
    trades = 0

    for i in range(len(signals)) :
        ts = ts_to_datetime(times[i]/1000)
        trade_profit = 0
        if market_position == 0 and df_signals['Buy'].iloc[i]:
            signals[i] = 1
            price = ticks[i] * buy_correction
            num_shares    = num_shares + contract
            open_position = num_shares * price*(1+commission)
            market_position = 1
            
        elif market_position == 1 and df_signals['Sell'].iloc[i]:
            signals[i] = -1
            price = ticks[i] * sell_correction
            close_position = num_shares * price * (1-commission)
            trade_profit = close_position - open_position
            cum_profit  += trade_profit
            num_shares   = 0
            market_position = 0
            trades = trades + 1

        #if signals[i] !=0:
        trade_statistics.append([ts, ticks[i], signals[i], market_position, num_shares, trade_profit, cum_profit, trades])            
        

    df = pd.DataFrame(trade_statistics)
    df.columns = ['time', 'price', 'signal', 'MP', 'num_shares', 'profit', 'cum_profit', 'trades']
    df = df.astype({'signal':np.int32, 'MP':np.int32})
    df['high_val'] = df['cum_profit'].cummax()
    df['drawdown'] = df['cum_profit'] - df['high_val']
    return df

In [5]:
xx = read_data('SOL-BTC', '1h', '2021-03-25')
xx.columns = (['T', 'open', 'high', 'low', 'close', 'volume'])
xx = xx.reset_index(drop=True)

In [7]:

CustomTrends = ta.Strategy(
    name="Trends",
    description="Trends",
    ta=[
        {"kind": "stoch"},#"fast":12, "slow":24},
    ]
)
# To run your "Custom Strategy"
xx.ta.strategy(CustomTrends)

In [32]:
xx['Buy'] = (xx.STOCHk_14_3_3 < 30) 
xx['Sell'] = (xx.STOCHk_14_3_3 > 70)
signals = xx[['Buy', 'Sell']]

prices = xx['close']

In [10]:
signals

,Buy,Sell
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
995,False,False
996,True,False
997,True,False
998,True,False


In [33]:
init_data = {'exchange_commission': 0.075, #percent
             'start_capital'     : 100,
             'buy_correction'    : 1.0001,
             'sell_correction'   : 0.9999,
             'contract'          : 1000,  
             'robot_name'        : '',
             'market'            : '',
             'kline'             : ''  
            }
res = simulation_01(xx, signals, init_data)


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(res[0:150])

In [34]:
res[res.signal == 1]

,time,price,signal,MP,num_shares,profit,cum_profit,trades,high_val,drawdown
15,2021-03-25 16:00:00,0.00024884,1,1,1000,0.0,0.000000000,0,0.000000000,0.000000000
31,2021-03-26 08:00:00,0.00024748,1,1,1000,0.0,0.003643513,1,0.003643513,0.000000000
104,2021-03-29 09:00:00,0.00032190,1,1,1000,0.0,0.015272547,2,0.015272547,0.000000000
137,2021-03-30 18:00:00,0.00033763,1,1,1000,0.0,0.019551212,3,0.019551212,0.000000000
182,2021-04-01 15:00:00,0.00032931,1,1,1000,0.0,0.016609255,4,0.019551212,-0.002941957
264,2021-04-05 01:00:00,0.00041090,1,1,1000,0.0,0.011323448,5,0.019551212,-0.008227764
352,2021-04-08 17:00:00,0.00045980,1,1,1000,0.0,0.013062844,6,0.019551212,-0.006488367
386,2021-04-10 03:00:00,0.00047546,1,1,1000,0.0,0.027308401,7,0.027308401,0.000000000
443,2021-04-12 12:00:00,0.00047089,1,1,1000,0.0,0.006237356,8,0.027308401,-0.021071046
464,2021-04-13 09:00:00,0.00044576,1,1,1000,0.0,0.003748279,9,0.027308401,-0.023560122
